# Introduction to Spark

## Objectives


You should be able to
 * Describe what Spark is
 * Describe what an RDD is
 * Explain now data is split over multiple machines
 * Write and execute RDD transformations

---
## Installation

Installing spark can be a challenge; we recommend you run it inside of a docker container, like this.

```
docker run --name spark -d -p 8880:8888 -v "$PWD":/home/jovyan/work ezamir/jupyter-spark-2.0
```
This may take a while to download. Note this is assuming port 8888 is being used (if probably is) and port 8880 is open. To access, point your browser to http://localhost:8880

You can install in directly on your Mac, but you may face challenges.
```
brew cask install homebrew/cask-versions/adoptopenjdk8
brew install apache-spark
```

# Key Concepts

## Spark is a tool for parallelized computation
- Highly efficient distributed operations
- Spark runs in memory and on disk
- Can be up to 100x faster than Hadoop MapReduce in memory, and 10x faster on disk.
- Spark keeps everything in memory when possible, uses lots of it.

<img src="images/spark_ecosystem.png" width="500">


## Parallelizing data storage: Resilient Distributed Datasets (RDD)

<img src="images/rdd_on_cluster.png" width="200" align="right">

[Image Source](http://horicky.blogspot.com/2015/02/big-data-processing-in-spark.html)

 
RDDs are the primary class introduced by Spark. It is a data container that allows for the construction of RDDs.

- **immutable** 
- **lazily evaluated**
- **cacheable**

## Parallelizing computation: a "functional programming paradigm" and DAGs

Spark provides many transformation functions. By programming these functions, you construct a **Directed Acyclic Graph** (DAG) of steps to execute the transformation.

<img src="images/dag.png">

When you use them, these functions are passed from the **client** to the **master**, who then distributes them to workers, who apply them across their partitions of the RDD.


## Spark architecture : from your coding hands to the cluster

<img src="images/from_rdd_to_cluster.png">

References: 
 - https://trongkhoanguyen.com/spark/understand-rdd-operations-transformations-and-actions/ 

You construct your sequence of transformations in python.

Spark functional programming interface builds up a **DAG**

This DAG is sent by the **driver** for execution to the **cluster manager**.

## Jargon

Excerpt taken from \[[Arush Kharbanda](https://www.quora.com/What-exactly-is-Apache-Spark-and-how-does-it-work) on Quora\]

**Job**: A piece of code which reads some input  from HDFS or local, performs some computation on the data and writes some output data.

**Stages**: Jobs are divided into stages. Stages are classified as a Map or reduce stages (It's easier to understand if you have worked on Hadoop and want to correlate). Stages are divided based on computational boundaries, all computations (operators) cannot be Updated in a single Stage. It happens over many stages.

**Tasks**: Each stage has some tasks, one task per partition. One task is executed on one partition of data on one executor(machine).

**DAG**: DAG stands for Directed Acyclic Graph, in the present context its a DAG of operators.

**Executor**: The process responsible for executing a task.

**Driver**: The program/process responsible for running the Job over the Spark Engine

**Master**: The machine on which the Driver program runs

**Slave/Worker**: The machine on which the Executor program runs

# Operational Spark in Python

<img src="images/spark_flow.png" width="500">

We'll proceed along the usual spark flow (see above).
1. create the enviromnent to run spark from python
2. extract RDDs from files
3. run some transformations
4. execute actions to obtain values (local objects in python)

**Brainstorming**: What kind of operations/actions might you want to do to your data?

- https://training.databricks.com/visualapi.pdf


put your ideas here...

##  Initializing a `SparkContext` in Python

`SparkContext` is the entry point to any spark functionality. When we run any Spark application, a driver program starts, which has the main function and your `SparkContext` gets initiated here.

The Jupyter notebook can be a *client* to interact with the *master*.

The client will have a `SparkContext` that..

1. Acts as a gateway between the client and Spark master
2. Sends code/data from Jupyter notebook to the master (who then sends it to the workers)

<img src="images/spark_driver_etc.png"/>

Using:

```python
import pyspark as ps
spark = ps.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext
```

will create a *"local"* cluster made of the driver using all 4 cores.

In [ ]:
import pyspark as ps    # for the pyspark suite

In [ ]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('lecture')
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
spark

## Practice Spark Job
Flip a coin 100 times using Python's `random()` function. What fraction of the time do you get heads?

In [ ]:
import random

In [ ]:
rdd = sc.parallelize(range(100)) # Distribute a local Python collection
                                 # to form an RDD.

In [ ]:
rdd

In [ ]:
rdd2 = rdd.map(lambda x: x**2)

In [ ]:
rdd2.take(5)

In [ ]:
# rdd.collect()

','.join(rdd.map(str).collect())

You can feed self-defined functions into `map` as well

In [ ]:
def my_square(x):
    return x**2

In [ ]:
(
rdd.map(my_square)
.map(lambda x: -x)
.filter(lambda x: x<-10)
.take(10)
)

In [ ]:
random.random()

In [ ]:
import numpy as np

np.random.seed(123)
n = 100
# heads = (sc.parallelize(range(n))
heads = (sc.parallelize(np.arange(n))
    .map(lambda _: random.random())
    .filter(lambda r: r < 0.5)
    .count())

tails = n - heads
ratio = heads / n

print('heads =', heads)
print('tails =', tails)
print('ratio =', ratio)

**Notes**
- `sc.parallelize` creates an RDD.
- `map` and `filter` are *transformations*.
  - They create new RDDs from existing RDDs.
- `count`, `take`, and `collect` are *actions* that bring the data from the RDDs back to the driver.

## Practice Spark job: Finding Primes

Q: Find all the primes less than 100.

- Define function to determine if a number is prime.

In [ ]:
def is_prime(number):
    ''' Return whether the given number is a prime number or not'''
    factor_min = 2
    factor_max = int(number ** 0.5) + 1
    for factor in range(factor_min, factor_max):
        if number % factor == 0:
            return False
    return True

- Use this to filter out non-primes.

In [ ]:
numbers = range(2, 100)

primes = (sc.parallelize(numbers)
    .filter(is_prime)
    .collect())

print(primes)

### Transformations and Actions

- Common RDD Constructors

Expression | Meaning
--- | ---
`sc.parallelize(list)` | Create RDD of elements of list
`sc.textFile(path)` | Create RDD of lines from file

- Common Transformations

Expression | Meaning
--- | ---
`filter(lambda x: x % 2 == 0)` | Discard non-even elements
`map(lambda x: x * 2)` | Multiply each RDD element by `2`
`map(lambda x: x.split())` | Split each string into words
`flatMap(lambda x: x.split())` | Split each string into words and flatten sequence
`sample(withReplacement = True, 0.25)` | Create sample of 25% of elements with replacement
`union(rdd)` | Append `rdd` to existing RDD
`distinct()` | Remove duplicates in RDD
`sortBy(lambda x: x, ascending = False)` | Sort elements in descending order

- Common Actions

Expression | Meaning
--- | ---
`collect()` | Convert RDD to in-memory list 
`take(3)` | First 3 elements of RDD 
`takeSample(withReplacement = True, 3)` | Create sample of 3 elements with replacement
`sum()` | Find element sum (assumes numeric elements)
`mean()` | Find element mean (assumes numeric elements)
`stdev()` | Find element deviation (assumes numeric elements)

## Creating an RDD (from files)

RDDs are **immutable**. Once created, you cannot modify them directly. You can only transform them into another RDD. 

Functions for creating an RDD from an external source are methods of the `SparkContext` object `sc`.

| Method | Description |
| :- | - |
| `sc.parallelize(array)` | Create an RDD from a python array or list |
| `sc.textFile(path)` | Create an RDD from a text file |
| `sc.pickleFile(path)` | Create an RDD from a pickle file |

Recall,  `sc.parallelize()` : create an RDD from a python iterable

In [ ]:
# creating an adhoc list
data_array = [['matthew', 4],
              ['jorge', 8],
              ['josh', 15],
              ['evangeline', 16],
              ['emilie', 23],
              ['yunjin', 42]]

# reading the array/list using SparkContext
rdd = sc.parallelize(data_array)

# to output the content in python [in practice, only use collect() with great care]
rdd.collect()

In [ ]:
# rdd.take?

**`sc.textFile()` : from a text file**

The import will give you an rdd made of **strings which are lines of the text file**.

In [ ]:
# displaying the content of the file in stdout
with open('data/toy_data.txt', 'r') as f:
    print(f.read())

In [ ]:
# reading the file using SparkContext
rdd = sc.textFile('data/toy_data.txt')

# to output the content in python [in practice, only use collect() with great care]
everything = rdd.collect()

In [ ]:
everything

### <span style="color:red">`sc.pickleFile()` : from a HDFS pickle file

The import will give you an rdd composed of whatever table was stored into that file.</span>

In [ ]:
%ls data/

In [ ]:
# reading the file using SparkContext
rdd = sc.pickleFile('data/toy_data.pkl')

# to output the content in python [irl, use with great care]
rdd.collect()

## Transformations : transforming an RDD into another

- They are **lazy**: Spark doesn't apply the transformation right away, it just builds on the **DAG**
- They transform an RDD into another RDD because RDD are **immutable**.
- They can be **wide** or **narrow** (whether they shuffle partitions or not).

<img src="images/rdd_narrow_vs_wide_transformations.png" width="400"/>

[Image Source](http://horicky.blogspot.com/2013/12/spark-low-latency-massively-parallel.html)



| Method | Type | Category | Description |
|:- |:- |:----:|:------------|
| [`.map(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) | transformation | mapping | Return a new RDD by applying a function to each element of this RDD. |
| [`.flatMap(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.flatMap) | transformation | mapping | Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results. |
| [`.filter(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.filter) | transformation | reduction |  Return a new RDD containing only the elements that satisfy a predicate. |
| [`.sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sample) | transformation | reduction | Return a sampled subset of this RDD. |
| [`.distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.distinct) | transformation | reduction |  Return a new RDD containing the distinct elements in this RDD. |
| [`.keys()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.keys) | transformation | `<k,v>` | Return an RDD with the keys of each tuple. |
| [`.values()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.values) | transformation | `<k,v>` | Return an RDD with the values of each tuple. |
| [`.join(rddB)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.join) | transformation | `<k,v>` | Return an RDD containing all pairs of elements with matching keys in self and other. Each pair of elements will be returned as a (k, (v1, v2)) tuple, where (k, v1) is in self and (k, v2) is in other. |
| [`.reduceByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) | transformation | `<k,v>` | Merge the values for each key using an associative and commutative reduce function. |
| [`.groupByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) | transformation | `<k,v>` | Merge the values for each key using non-associative operation, like mean. |
| [`.sortBy(keyfunc)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortBy) | transformation | sorting |  Sorts this RDD by the given keyfunc. |
| [`.sortByKey()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sortByKey) | transformation | sorting/`<k,v>` | Sorts this RDD, which is assumed to consist of (key, value) pairs. |



## Map vs. FlatMap

Here's the difference between `map` and `flatMap`:

- Map:

In [ ]:
!cat input.txt

In [ ]:
(
sc.textFile('input.txt')
    .map(lambda x: x.split())
    .collect()
)

- FlatMap:

In [ ]:
sc.textFile('input.txt')

In [ ]:
sc.textFile('input.txt').flatMap(lambda x: x.split())

In [ ]:
(sc.textFile('input.txt') 
    .flatMap(lambda x: x.split()) 
    .collect())

### Applying transformations and chaining them

Recall the spark flow:

<img src="images/spark_flow.png" width="500">

In the sequence below, we will in one sequence:
1. read an RDD from a text file
2. transform by applying `split`
3. transform by filtering
4. transform by casting some columns to their corresponding type
5. use an action to output the results

Each transformation is a method of an RDD, and returns another RDD.

In [ ]:
# displaying the content of the file in stdout
with open('data/sales.txt', 'r') as f:
    print(f.read())

*Recall: Input functions, reading RDDs from files, are functions of the SparkContext.*

In [ ]:
# reads a text file line by line
rdd1 = sc.textFile('data/sales.txt')

rdd1.collect()

In [ ]:
rdd1.sample(False, 0.5).collect()

In [ ]:
# applies split() to each row
rdd2 = rdd1.map(lambda rowstr : rowstr.split())

rdd2.collect()

In [ ]:
# filters rows
rdd3 = rdd2.filter(lambda row: not row[0].startswith('#'))

rdd3.collect()

In [ ]:
'foo'.startswith('F')

In [ ]:
def casting_function(row):
    ''' assert data type for each column '''
    id_, date, store, state, product, amount = row
    return (int(id_), date, int(store), state, int(product), float(amount))

# applies casting_function to rows
rdd4 = rdd3.map(casting_function)

# shows the result
rdd4.collect()

**Now, let's see the canonical way to write that in Python...**

In [ ]:
rdd_sales = sc.textFile('data/sales.txt')

rdd_sales.collect()

In [ ]:
rdd_sales = (sc.textFile('data/sales.txt')
        .map(lambda rowstr : rowstr.split()))   # <= JUST ADDED THIS HERE

rdd_sales.collect()

In [ ]:
rdd_sales = (sc.textFile('data/sales.txt')
        .map(lambda rowstr : rowstr.split())
        .filter(lambda row: not row[0].startswith('#')) )   # <= JUST ADDED THIS HERE

rdd_sales.collect()

In [ ]:
rdd_sales = (sc.textFile('data/sales.txt')
        .map(lambda rowstr : rowstr.split())
        .filter(lambda row: not row[0].startswith('#'))
        .map(casting_function) )  # <= JUST ADDED THIS HERE

rdd_sales.collect()

<span style="color:red">FROM NOW ON WE'LL RELY ON THESE TWO RDDs</span>

In [ ]:
# creating an adhoc list
data_array = [['matthew', 4],
              ['jorge', 8],
              ['josh', 15],
              ['evangeline', 16],
              ['emilie', 23],
              ['yunjin', 42]]

# reading the array/list using SparkContext
rdd_names = sc.parallelize(data_array)

# to output the content in python [irl, use with great care]
rdd_names.collect()

In [ ]:
rdd_sales = (sc.textFile('data/sales.txt')
        .map(lambda x : x.split())
        .filter(lambda x: not x[0].startswith('#'))
        .map(casting_function))

rdd_sales.collect()

### Mapping

#### `.map(func)` : applying a function on every row

In [ ]:
# applying a lambda function to an rdd
rddout = rdd_names.map(lambda x : len(x[0]))

# print out the original rdd
print("before: {}".format(rdd_names.collect()))

# print out the new rdd generated
print("\nafter: {}".format(rddout.collect()))

#### `.flatMap(func)` : applying a function on every row and flattening the resulting lists

In [ ]:
#rdd.flatMap?

In [ ]:
# applying a lambda function to an rdd (because why not)
dumb_function = lambda x : (x[1], x[1]+2, x[1]+len(x[0]) )

# print out the original rdd
print("before: {}".format(rdd_names.collect()))

print("\njust map: {}".format(rdd_names.map(dumb_function).collect()))
# print out the new rdd generated
print("\njust flatmap: {}".format(rdd_names.flatMap(dumb_function).collect()))

### Row reduction

#### `.filter(func)`: filters an RDD using a function that returns boolean values

In [ ]:
# filtering an rdd
rddout = rdd_sales.filter(lambda row: (row[3] == 'CA'))

# print out the original rdd
print("\nbefore: {}".format(rdd_sales.collect()))

# print out the new rdd generated
print("\nafter: {}".format(rddout.collect()))

#### `.sample(withReplacement, fraction, seed)`: sampling an RDD

In [ ]:
# sampling an rdd
rddout = rdd_sales.sample(True, 0.4)

# print out the original rdd
print("\nbefore: {}".format(rdd_sales.collect()))

# print out the new rdd generated
print("\nafter: {}".format(rddout.collect()))

#### `.distinct()`: obtaining distinct rows

In [ ]:
# obtaining distinct values of the "state" column of rdd_sales
rddout = (rdd_sales.map(lambda row: row[3])\
          .distinct())

# print out the original rdd
print("before: {}".format(rdd_sales.collect()))

# print out the new rdd generated
print("\nafter: {}".format(rddout.collect()))

### Methods with a `<k,v>` paradigm

#### `.values()`: returns the values of a RDD made of `<k,v>` pairs

In [ ]:
# applying a lambda function to an rdd (because why not)
rddout = rdd_names.values()

# print out the original rdd
print("before: {}\n".format(rdd_names.collect()))
# print out the new rdd generated
print("after: {}".format(rddout.collect()))

In [ ]:
rdd_sales.values().collect()

#### `.keys()`: returns the keys of a RDD made of `<k,v>` pairs

In [ ]:
# applying a lambda function to an rdd (because why not)
rddout = rdd_names.keys()

# print out the original rdd
print("before: {}\n".format(rdd_names.collect()))

# print out the new rdd generated
print("after: {}".format(rddout.collect()))

In [ ]:
# applying a lambda function to an rdd (because why not)
rddout = rdd_names.map(lambda x : [x[1], x[1]+2, x[1]+len(x[0])])
# print out the original rdd
print("before: {}\n".format(rdd_names.collect()))

# print out the new rdd generated
print("after: {}".format(rddout.collect()))

In [ ]:
# applying a lambda function to an rdd (because why not)
rddout = rdd_names.map(lambda x : [x[1], x[1]+2, x[1]+len(x[0])]).keys()
# print out the original rdd
print("before: {}\n".format(rdd_names.collect()))

# print out the new rdd generated
print("after: {}".format(rddout.collect()))

#### `rddA.join(rddB)`: join another RDD

In [ ]:
rdd_salesperstate = rdd_sales.map(lambda row: (row[3],row[-1]))

rdd_salesperstate.collect()

In [ ]:
# creating an adhoc list of managers for each state
data_array = [['CA', 'matthew'],
              ['OR', 'jorge'],
              ['WA','matthew'],
              ['TX', 'emilie']]

# reading the array/list using SparkContext
rdd_managers = sc.parallelize(data_array)

# to output the content in python [irl, use with great care]
rdd_salesperstate.join(rdd_managers).collect()

#### `.reduceByKey(func)`: reduce `v`s by their `k` by applying func (what ?)

The `func` here needs to be associative and commutative... 

In [ ]:
# creating an adhoc list
data_array = [['CA', 1],
              ['WA', 1],
              ['CA', 2],
              ['OR', 1],
              ['CA', 5],
              ['OR', 1]]

# reading the array/list using SparkContext
rdd = sc.parallelize(data_array)

# to output the content in python [irl, use with great care]
rdd.collect()

In [ ]:
rdd.reduceByKey(lambda v1,v2 : (v1*v2)).collect()

#### `.groupByKey(func)`: reduce `v`s by their `k` by applying func (again ?)

This can use any function non-commutative

In [ ]:
# creating an adhoc list
data_array = [['CA', 1],
              ['WA', 1],
              ['CA', 2],
              ['OR', 1],
              ['CA', 5],
              ['OR', 1]]

# reading the array/list using SparkContext
rdd = sc.parallelize(data_array)

# to output the content in python [irl, use with great care]
rdd.collect()

In [ ]:
def mean(iterator):
    total = 0.0
    count = 0
    for x in iterator:
        total += x
        count += 1
    return total / count

(rdd.groupByKey()
    .map(lambda x: (x[0], mean(x[1])))
    .collect())

### Sorting methods

#### `.sortBy(keyfunc)`: sorting by the value of a function on rows

In [ ]:
# sorting by any function (because why not?)
rddout = rdd_names.sortBy(lambda x : (13-x[1])**2, ascending=True)
# print out the original rdd
print(rdd_names.collect(), "\n")
# print out the new rdd generated
print(rddout.collect())

# rdd_names.map(lambda x: (x[0], (13-x[1])**2)).collect()

In [ ]:
# sorting k,v pairs by key
rddout = rdd_names.sortByKey(ascending=False)
# print out the original rdd
print(rdd_names.collect(), "\n")
# print out the new rdd generated
print(rddout.collect())

## Actions : turning your RDD into something else (local object)

Actions are specific methods of an RDD object, they are usually designed to transform an RDD into something else (a python object, or a statistic).

When used/executed in IPython or in a notebook, they **launch the processing of the DAG**. This is where Spark stops being **lazy**. This is where your script will take time to execute.

| Method | Type | Description |
| - | - | - |
| [`.collect()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) | action | Return a list that contains all of the elements in this RDD. Note that this method should only be used if the resulting array is expected to be small, as all the data is loaded into the driver’s memory. |
| [`.count()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.count) | action | Return the number of elements in this RDD. |
| [`.take(n)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take) | action | Take the first `n` elements of the RDD. |
| [`.top(n)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.top) | action | Get the top `n` elements from an RDD. It returns the list sorted in descending order. |
| [`.first()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.first) | action | Return the first element in an RDD. |
| [`.sum()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sum) | action | Add up the elements in this RDD. |
| [`.mean()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.mean) | action | Compute the mean of this RDD’s elements. |
| [`.stdev()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.stdev) | action | Compute the standard deviation of this RDD’s elements. |

In [ ]:
# creating an adhoc list
data_array = [['matthew', 4],
              ['jorge', 8],
              ['josh', 15],
              ['evangeline', 16],
              ['emilie', 23],
              ['yunjin', 42]]

# reading the array/list using SparkContext
rdd_names = sc.parallelize(data_array)

### Actions that return portions of an RDD

#### `.collect()` : returning the *full* content of an RDD to "python space"

Returns the rows of an RDD as a list. Can be a bad idea if your RDD is gigantic, cause `.collect()` will return everything and put it in memory for python to process.

In [ ]:
# to output the content in python
collected = rdd_names.collect()

# let's check the type of RDD
print("type of rdd: {}\n".format(type(rdd_names)))
# let's check the type of what's collected
print("type of rdd_collected: {}\n".format(type(collected)))
# let's print the collected content
print(collected)

#### `.take(n)` : returning (any) $n$ lines of an RDD

Returns `n` the rows of an RDD as a list. These `n` are not randomly selected. They are Spark's own internal mechanism for obtaining the lines that can be collected first.

In [ ]:
# to output the content in python
taken = rdd_names.take(2)
# let's check the type of what's collected
print("type of rdd_taken: {}\n".format(type(taken)))
# let's print the collected content
print(taken)

#### `.first()` : returning the first line of an RDD

In [ ]:
print(rdd_names.first()) ## actually call take(1) in source code

### Actions that compute some statistics

#### `.count()` : count the number of lines

In [ ]:
print(rdd_names.count())

#### `.sum()`: summing every line in an RDD

(The RDD needs to be containing summable values)

In [ ]:
print(rdd_names.values().sum())

#### `.mean()`: averaging every line in an RDD

(The RDD needs to be containing summable values)

In [ ]:
print(rdd_names.values().mean())

#### `.stdev()`: you get that right ?

In [ ]:
print(rdd_names.values().stdev())

## Let's design chains of transformations together!

### Computing sales per state
Input RDD

In [ ]:
rdd_sales = (sc.textFile('data/sales.txt')
        .map(lambda x : x.split())
        .filter(lambda x: not x[0].startswith('#'))
        .map(casting_function)
            )
# rdd_sales
rdd_sales.collect()

### Task

You want to obtain a sorted RDD of the states in which you have most sales done (amount).

What transformations do you need to apply ?
If you had to draw a workflow of the transformations to apply ?

### Code

In [ ]:
## YOU DO! 
# apply transformation here...

# rdd_sales.




<span style="color:white;font-family:'Courier New'"><br/>
Solution <br>
rdd_sales.map(lambda x: (x[3],x[5])).top(1) <br>
rdd_sales.map(lambda x: (x[3], x[5])).groupByKey().mapValues(sum).top(1)
</span>

## Word count (again)

### Input RDD

In [ ]:
# displaying the content of the file in stdout
with open('data/input.txt', 'r') as f:
    print(f.read())

# reading the file using SparkContext
rdd = sc.textFile('data/input.txt')

### Task
What transformations do you need to apply ?
If you had to draw a workflow of the transformations to apply ?

### You Do!

In [ ]:
rddout = rdd # apply transformation here...

# collect the result
# rddout.collect()

<span style="color:white;font-family:'Courier New'">
rddout = rdd.flatMap(lambda str : str.split())\<br/>
            .map(lambda word: (word,1))\<br/>
            .reduceByKey(lambda v1,v2: v1+v2)<br/>
<br/>
rddout.collect()<br/>
</span>

## Find the date on which AAPL's stock price was the highest

### Input RDD

In [ ]:
rdd_aapl_raw = sc.textFile('data/aapl.csv')

print("lines in file: {}".format(rdd_aapl_raw.count()))

rdd_aapl_raw.take(5)

### Task

Now, design a pipeline that would :
1. filter out headers
2. split each line based on comma
3. keep only fields for Date (col 0) and Close (col 4)
4. order by Close in descending order

### You do!

In [ ]:
rddout = rdd_aapl_raw # apply transformation here...

# rddout.take(5)

<span style="color:white;font-family:'Courier New'">
rddout = rdd_aapl_raw.filter(lambda line: not line.startswith("Date"))\<br/>
.map(lambda line: line.split(","))\<br/>
.map(lambda fields: (float(fields[4]),fields[0]))\<br/>
.sortBy(lambda (close, date): close, ascending=False)
<br/>
rddout.collect()<br/>
</span>


# Caching / Persistency

- The RDD does no work until an action is called. And then when an action is called it figures out the answer and then throws away all the data.
- If you have an RDD that you are going to reuse in your computation you can use cache() to make Spark cache the RDD.
- This is especially useful if you have to run the same computation over and over again on one RDD: one use case ? oh I don't know maybe... **MACHINE LEARNING!**

## Caching

Consider the following job...

In [ ]:
import random
import math
np.random.seed(999)

num_count = 10**7
num_list = [random.random() for i in range(num_count)]
rdd1 = sc.parallelize(num_list)
rdd2 = rdd1.sortBy(lambda num: num)

In [ ]:
%time rdd2.count()
%time rdd2.count()
%time rdd2.count()

- Lets cache it and try again.

In [ ]:
rdd2.cache()
%time rdd2.count()
%time rdd2.count()
%time rdd2.count()

- **theoretically** Caching the RDD speeds up the job because the RDD does not have to be computed from scratch again.
- Calling cache() flips a flag on the RDD.
- The data is not cached until an action is called.
- You can uncache an RDD using unpersist()

## Persist [won't go into details]

- Persist RDD to disk instead of caching it in memory.
- You can cache RDDs at different levels.
[more details](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-rdd-caching.html)

| Level	| Meaning |
| - | - |
| MEMORY_ONLY	| Same as cache() |
| MEMORY_AND_DISK	| Cache in memory then overflow to disk |
| MEMORY_AND_DISK_SER	| Like above; in cache keep objects serialized instead of live |
| DISK_ONLY	| Cache to disk not to memory |